In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [3]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
import versor as vsr
from versor.drawing import *

In [5]:
from IPython.display import display
from ipywidgets import HTML, Text, interact, widgets
from traitlets import link, dlink

In [6]:
def rgb2hex(r, g, b):
    return "#{0:02x}{1:02x}{2:02x}".format(r, g, b)

# Figures

## Reciprocal frame

In [47]:
R = vsr.Biv(pi/6.,0,0).exp()
a1 = vsr.Vec(0.5,-1,0)
a2 = vsr.Vec(1.,1.,0)
a3 = vsr.Vec(-1,-1,1)

A = (a1 ^ a2 ^ a3)

print(A.inv())

ra1 = (a2 ^ a3) <= A.inv()
ra2 = (a3 ^ a1) <= A.inv()
ra3 = (a1 ^ a2) <= A.inv()

print(ra1)

b = 1 * a1 + 2. * a2 + 1. * a3
# print(b)
coords = np.array([(b <= ra1),(b <= ra2), (b <= ra3)])
# print(np.array([(b <= ra1),(b <= ra2), (b <= ra3)]))
print(coords[0] * a1)

scene = Scene(children=[
#         frame_mesh(),
        vector_mesh(a1, color='red', arrow=True),
        vector_mesh(a2, color='green', arrow=True),
        vector_mesh(a3, color='blue', arrow=True),
        vector_mesh(a1, position=a2, color='gray'),
        vector_mesh(a1, position=a3, color='gray'),
        vector_mesh(a1, position=a3+a2, color='gray'),
        vector_mesh(a2, position=a1, color='gray'),
        vector_mesh(a2, position=a3, color='gray'),
        vector_mesh(a2, position=a3+a1, color='gray'),
        vector_mesh(a3, position=a1, color='gray'),
        vector_mesh(a3, position=a2, color='gray'),
        vector_mesh(a3, position=a1+a2, color='gray'),
        vector_mesh(ra1, color='hotpink', arrow=True),
        vector_mesh(ra2, color='lightgreen', arrow=True),
        vector_mesh(ra3, color='lightskyblue', arrow=True),
#         vector_mesh((coords[0] * a1), position=b, color='gray'),
#         vector_mesh(b-a1, position=a1),
#         vector_mesh(b-ra1, position=ra1),
#         vector_mesh(b),

        AmbientLight(color='#cccccc')])
camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'

display(renderer)

Tri: [ -0.67 ]
Vec: [ 0.67 -0.67 0 ]
[ 0.5 -1.   0. ]


## Duality

In [26]:
B = vsr.Biv(1,0,0)
a = B.duale()

scene = Scene(children=[
        frame_mesh(),
        vector_mesh(a,color='blue', arrow=True),
        bivector_mesh(B,as_plane=False,color='hotpink'),
#         DirectionalLight(color='#ccaabb', position=[1,1,1]),
        AmbientLight(color='#cccccc')])
camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'

display(renderer)

## Projection and Rejection

In [8]:
a = vsr.Vec(1,1,1)
B = vsr.Biv(1,0,0)
c = a.project_onto(B)
d = a.reject_from(B)

scene = Scene(children=[
        frame_mesh(),
        vector_mesh(a,color='gray', arrow=True),
        vector_mesh(c,color='gray'),
        vector_mesh(d,position=np.array(c).tolist(),color='gray'),
        bivector_mesh(B,color='hotpink'),
#         DirectionalLight(color='#ccaabb', position=[1,1,1]),
        AmbientLight(color='#cccccc')])
camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

## Reflection

In [9]:
a = vsr.Vec(1,0,1)
B = vsr.Biv(2,0,0)
n = B.duale().unit()
a_reflected = a.reflect_in_line(n)
a_reflected2 = a.reflect_in_plane(n)

children = [
    frame_mesh(),
    vector_mesh(a,color='black', arrow=True),
    vector_mesh(a_reflected,color='black', arrow=True),
    vector_mesh(a_reflected2,color='black', arrow=True),
#     vector_mesh(n,color='black', arrow=True),
    bivector_mesh(B, as_plane=True,color='hotpink')]

children[-1].material.transparent = True
children[-1].material.opacity = 0.75


def show(children):
    scene = Scene(children= children + [AmbientLight(color='#cccccc')])
    camera = PerspectiveCamera(position=[1, -3, 0])
    renderer = Renderer(camera=camera, 
                        scene=scene, 
                        controls=[TrackballControls(controlling=camera)],
                        background = 'white')
    return renderer
display(show(children))

## Rotation

In [10]:
n1 = vsr.Vec(0,1,0).unit()
n2 = n1.spin(vsr.Biv(-np.deg2rad(30) / 2,0,0).exp())
B = n1 ^ n2
R = n1 * n2
a = vsr.Vec(1,0,1).unit()
a_rot = a.reflect_in_line(n2).reflect_in_line(n1)

B_mesh = bivector_mesh(B,color='gray')
B_mesh.material.opacity = 0.9
B_mesh.material.transparent = True

scene = Scene(children=[
#         frame_mesh(),
        vector_mesh(n1,color='yellow'),
        vector_mesh(n2,color='cyan'),
        vector_mesh(a, color='hotpink'),
        vector_mesh(a.reflect_in_line(n2)),
        vector_mesh(a_rot, color='black'),
#         bivector_mesh(n1.unduale(),color='yellow'),
#         bivector_mesh(n2.unduale(),color='cyan'),
        B_mesh,
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

# Conformal Geometric Algebra

In [11]:
p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(0,1,1).null()

scene = Scene(children=[
        frame_mesh(),
        point_mesh(p1,color='lightskyblue'),
        vector_mesh(p1.vec(),color='lightslategray', arrow=True),
        point_mesh(p2,color='lightskyblue'),
        vector_mesh(p2.vec(),color='lightslategray', arrow=True),

        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

In [12]:
# p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(0,1,1).null()
p3 = vsr.Vec(1,1,1).null()
p4 = vsr.Vec(0,0,1.5).null()
pln = vsr.Pln(p2, p3, p4).dual()


scene = Scene(children=[
        frame_mesh(),
        point_mesh(p2,color='lightskyblue'),
        vector_mesh(p2.vec(),color='lightskyblue', arrow=True),

        point_mesh(p3,color='lightskyblue'),
        point_mesh(p4,color='lightskyblue'),
        plane_mesh(pln, width=3, height=3, color='hotpink'),

        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

In [13]:
# p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(0,1,1).null()
p3 = vsr.Vec(1,1,1).null()
p4 = vsr.Vec(0,0,1.5).null()
pln = vsr.Pln(p2, p3, p4).dual().unit()

loc = pln.loc(vsr.Vec(0,0,0).null()).vec()
n = loc - vsr.Vec(0,0,0)
print(n)


scene = Scene(children=[
        frame_mesh(),
#         point_mesh(loc,color='lightskyblue'),
        plane_mesh(pln, width=3, height=3, color='hotpink'),
        vector_mesh(n),
        vector_mesh(n,position=loc,arrow=True),
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

Vec: [ 0 0.6 1.2 ]


In [14]:
n

Vec: [ 0 0.6 1.2 ]

### Circle

In [15]:
# p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(0,1,1).null()
p3 = vsr.Vec(1,1,1).null()
p4 = vsr.Vec(0,0,1.5).null()
cir = vsr.Cir(p2, p3, p4)


scene = Scene(children=[
        frame_mesh(),
        point_mesh(p2,color='lightskyblue'),
        point_mesh(p3,color='lightskyblue'),
        point_mesh(p4,color='lightskyblue'),
        circle_mesh(cir, color='coral'),

        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

In [16]:
vsr.Sph?

### Direct Line

In [17]:
p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(2,1,1).null()
l = vsr.Dll(p1,p2).unit()
print(l)
dlp = vsr.Dlp(l.dir().vec().unit(), 0)

ld = vsr.Vec(*np.array(l)[3:])

l_dir = l.dir().vec().unit()

l_loc = l.loc(vsr.Vec(0,0,0).null()).vec()

d = l_loc <= l_dir.unduale()

vertices = [[0,0,0], 
            np.array(p1.vec()).tolist(),
            np.array(p2.vec()).tolist(),
            np.array(p2.vec() - p1.vec()).tolist() 
           ]
faces= [[1,2,3],[1,4,3]]

scene = Scene(children=[
        frame_mesh(),
        point_mesh(p1,color='#ff00ff'),
        vector_mesh(p1.vec(),color='#ff00ff'),
#         plane_mesh(dlp, color='plum'),
        Mesh(geometry=PlainGeometry(vertices=vertices, faces=faces),
             material=LambertMaterial(color='palegreen',
                                      transparent=True,
                                      opacity=0.75)),
        
        point_mesh(p2,color='#00ffff'),
#         vector_mesh(l_dir,position=p1,color='#ffff00', arrow=True),
#         bivector_mesh(l.biv(), color='#ffff00'),
#         bivector_mesh(p1.vec() ^ p2.vec(), color='coral', as_plane=True),
#         vector_mesh(l.drv().vec(), color='coral'),
#         point_mesh(l.meet(dlp), color='red'),

        line_mesh(l, length=100),
        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

Dll: [ 0 -0.7071 0.7071 -0.7071 0.7071 0.7071 ]


### Dual Line

In [18]:
p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(2,1,1).null()
l = vsr.Dll(p1,p2).unit()
print(l)
dlp = vsr.Dlp(l.dir().vec().unit(), 0)

ld = vsr.Vec(*np.array(l)[3:])

l_dir = l.dir().vec().unit()

l_loc = l.loc(vsr.Vec(0,0,0).null()).vec()

d = l_loc <= l_dir.unduale()

vertices = [[0,0,0], 
            np.array(p1.vec()).tolist(),
            np.array(p2.vec()).tolist(),
            np.array(p2.vec() - p1.vec()).tolist() 
           ]
faces= [[1,2,3],[1,4,3]]

scene = Scene(children=[
        frame_mesh(),
        point_mesh(p1,color='#ff00ff'),
        vector_mesh(p1.vec(),color='#ff00ff'),
#         plane_mesh(dlp, color='plum'),
#         Mesh(geometry=PlainGeometry(vertices=vertices, faces=faces),
#              material=LambertMaterial(color='palegreen',
#                                       transparent=True,
#                                       opacity=0.75)),
        
        point_mesh(p2,color='#00ffff'),
#         vector_mesh(l_dir,position=p1,color='#ffff00', arrow=True),
        bivector_mesh(l.biv(), color='lightblue'),
#         bivector_mesh(p1.vec() ^ p2.vec(), color='coral', as_plane=True),
        vector_mesh(l.drv().vec(), color='deepskyblue'),
#         point_mesh(l.meet(dlp), color='red'),

        line_mesh(l, length=100),
        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

Dll: [ 0 -0.7071 0.7071 -0.7071 0.7071 0.7071 ]


### Tangent vector

In [19]:
p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(2,1,1).null()
l = vsr.Dll(p1,p2).unit()
print(l)
dlp = vsr.Dlp(l.dir().vec().unit(), 0)

ld = vsr.Vec(*np.array(l)[3:])

l_dir = l.dir().vec().unit()

l_loc = l.loc(vsr.Vec(0,0,0).null()).vec()

d = l_loc <= l_dir.unduale()

vertices = [[0,0,0], 
            np.array(p1.vec()).tolist(),
            np.array(p2.vec()).tolist(),
            np.array(p2.vec() - p1.vec()).tolist() 
           ]
faces= [[1,2,3],[1,4,3]]

scene = Scene(children=[
        frame_mesh(),
        point_mesh(p1,color='#ff00ff'),
        vector_mesh(p1.vec(),color='#ff00ff'),
        vector_mesh((p2.vec() - p1.vec()).unit(), position=p1.vec(), arrow=True),
        vector_mesh((p2.vec() - p1.vec()).unit(), arrow=True),

        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

Dll: [ 0 -0.7071 0.7071 -0.7071 0.7071 0.7071 ]


### Carrier flats

In [20]:
# p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(0,1,1).null()
p3 = vsr.Vec(1,1,1).null()
p4 = vsr.Vec(0,0,1.5).null()
cir = vsr.Cir(p2, p3, p4)
pln = vsr.Pln(p2,p3,p4)


scene = Scene(children=[
        frame_mesh(),
#         point_mesh(p2,color='lightskyblue'),
#         point_mesh(p3,color='lightskyblue'),
#         point_mesh(p4,color='lightskyblue'),
        circle_mesh(cir, color='black'),
        plane_mesh(pln, width=3, height=3, color='lightskyblue'),

        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

### Transformations

#### Transformations

In [21]:

normal = vsr.Vec(1,1,1).unit()
pao = vsr.Vec(1,0,0).null()
pln1 = vsr.Dlp(normal,1.)
pln2 = vsr.Dlp(normal,1.75)

p1 = pao.reflect_in_plane(pln1)
p2 = p1.reflect_in_plane(pln2)

scene = Scene(children=[
        frame_mesh(),
        plane_mesh(pln1, color='hotpink', width=2., height=2.),
        plane_mesh(pln2, color='hotpink', width=2., height=2.),
        
        point_mesh(pao, color='lightskyblue'),
        point_mesh(p1, color='lightskyblue'),
        point_mesh(p2, color='lightskyblue'),

#         point_mesh(pao.spin(T)),
        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

#### General Rotations

In [87]:
t = vsr.Vec(0,1,1)
T = t.trs()
R = vsr.Biv(0,0,-pi/6).exp()
B = R.log().unit()
G = T * R * T.rev()
print(G)

p = vsr.Vec(1,1,2).null()
ptrsrev = p.spin(T.rev())
prottrsrev = ptrsrev.spin(R)
pfinal = prottrsrev.spin(T)

line = G.log().unit()

v1 = p.vec() - line.loc(p).vec() 
v2 = pfinal.vec() - line.loc(p).vec()

scene = Scene(children=[
        frame_mesh(),
        vector_mesh(t,color='black', arrow=True),
        point_mesh(p, color='yellow'),
        point_mesh(ptrsrev, color='orange'),
        point_mesh(prottrsrev, color='hotpink'),
        point_mesh(p.spin(G), color='deepskyblue'),
        line_mesh(line, length=100),
        vector_mesh((t <= B), color='deepskyblue'),
        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

Mot: [ 0.87 0 0 0.5 0 0.5 -0.5 0 ]


####  Motor and Screws

In [96]:
t = vsr.Vec(0,1,1)
T = t.trs()
R = vsr.Biv(0,0,-pi/6).exp()
B = R.log().unit()
G = T * R * T.rev()
print(G)

p = vsr.Vec(1,1,2).null()
ptrsrev = p.spin(T.rev())
prottrsrev = ptrsrev.spin(R)
ptrt = prottrsrev.spin(T)

tpar = R.log().unit()
Tpar = tpar.duale().trs()
pfinal = ptrt.spin(Tpar)

w = pfinal.vec() - ptrt.vec()

line = G.log().unit()

pc = line.loc(p).vec() 
v1 = p.vec() - pc 
v2 = ptrt.vec() - pc

m1 = vsr.Vec(1,1,1).trs() * vsr.Biv(0,0,-pi/6).exp()
m1log = m1.log().unit()
m2 = Tpar*vsr.Rot(1,0,0,0) * G
m2log = m2.log().unit()

mi = m2log.ratio(m1log,0.2)


scene = Scene(children=[
        frame_mesh(),
        vector_mesh(t,color='gray', arrow=True),
        point_mesh(p, color='darkturquoise'),
        point_mesh(ptrt, color='deepskyblue'),
        point_mesh(pfinal, color='dodgerblue'),
        vector_mesh(w, position=ptrt.vec(), color='gray', arrow=True),
        bivector_mesh(B, color='lightpink'),
        line_mesh(line, length=100),
        
        vector_mesh(v1, position=pc, color='gray'),
        vector_mesh(v2, position=pc, color='gray'),
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

Mot: [ 0.87 0 0 0.5 0 0.5 -0.5 0 ]


In [81]:
mi = m1log.ratio(m2log,0.2)
mi

Mot: [ 1 0 0 0 0 0.12 -0.087 0 ]

In [82]:
vsr.Vec(0,0,0).null().spin(m1 * mi)

Dls: [ 1 1.027 2.3 1 3.672 ]

In [97]:
m2

Mot: [ 0.87 0 0 0.5 -0.43 0.5 -0.5 -0.25 ]

In [98]:
m2.rev()

Mot: [ 0.87 -0 -0 -0.5 0.43 -0.5 0.5 -0.25 ]

In [62]:
m1.log()

Dll: [ 0 0 0 -0.5 -0.5 -1 ]

In [63]:
m2.log()

Dll: [ 0 0 0.5236 -0.5 0.5236 -0.5236 ]

In [23]:
print(pln2.loc(pao))

Dls: [ 1.677 0.677 0.677 1 2.552 ]


In [8]:

l1 = vsr.Dll(vsr.Vec(0,0,1).null(), vsr.Vec(1,1,1).null()).unit()
l = vsr.Dll(vsr.Vec(0,0.,1).null(), vsr.Vec(1,0,1).null()).unit()

p1 = vsr.Vec(0,0,0).null().reflect_in_line(l1.inv())
p2 = vsr.Vec(0,0,0).null().reflect_in_line(l1.inv()).reflect_in_line(l)

mot = l.ratio(l1,1.0)
pao = vsr.Vec(0,0,0).null()

scene = Scene(children=[
        frame_mesh(),

        point_mesh(p1, color='magenta'),
        point_mesh(p2, color='cyan'),
        line_mesh(l1, color='red', length=100),

        line_mesh(l1.inv(), color='magenta', length=100),
        line_mesh(l, color='cyan', length=100),
#         line_mesh( color='cyan', length=100),
        Line(geometry=PlainGeometry(vertices=[[0,0,0], np.array(p1.vec()).tolist()],
                                   colors = ['magenta', 'magenta']), 
            material=LineBasicMaterial(linewidth=1,
                                       vertexColors='VertexColors'), 
            type='LinePieces'),
        Line(geometry=PlainGeometry(vertices=[np.array(p1.vec()).tolist(), np.array(p2.vec()).tolist()],
                                   colors = ['cyan', 'cyan']), 
            material=LineBasicMaterial(linewidth=1,
                                       vertexColors='VertexColors'), 
            type='LinePieces'),
        
        point_mesh(pao.spin(mot), color='hotpink'),
        line_mesh(l.spin(mot)),
        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

In [25]:
print(p2)

Pnt: [ -0.6667 0.6667 0.6667 1 0.6667 ]


In [26]:
print(p2)

Pnt: [ -0.6667 0.6667 0.6667 1 0.6667 ]


In [27]:
M = ((l * l1.inv()).log() / 2.).exp()
print(M)

Mot: [ 0.89 0.33 0.33 0 0.33 0 0 0 ]


In [28]:
print(vsr.Vec(0,0,0).null().spin(l * l1.inv()))

Pnt: [ -0.6667 0.6667 0.6667 1 0.6667 ]


In [29]:
print(vsr.Vec(0,0,0).null().reflect_in_line(l1.inv()).reflect_in_line(l))

Pnt: [ -0.6667 0.6667 0.6667 1 0.6667 ]


### Lines

In [10]:
p1 = vsr.Vec(1,0,1).null()
p2 = vsr.Vec(2,1,1).null()

l1 = vsr.Dll(vsr.Vec(0,0,0).null(), vsr.Vec(1,0,0).null()).unit()
# l = vsr.Dll(vsr.Vec(0,0.,1).null(), vsr.Vec(1,0,1).null()).unit()

l = vsr.Dll(p1,p2).unit()
l_dir = l.dir().vec().unit()
l_loc = l.loc(vsr.Vec(0,0,0).null()).vec()

l2 = l - l1
l3 = l.comm(l1)
print(l3)

m1 = l1.ratio(l,0.1)
# print(l)
# print(l1.spin(m1))
# ms = [m1]
# for i in range(9):
#     m1 = m1 * m1
#     ms.append(m1)
    
# print(ms)
    

# line_meshes = [line_mesh(l) for ] 

m = ((l * l1.inv()).log() * 0.5 ).exp()
w = m.trs().reject_from(m.rot().log().unit())
v = m.trs().project_onto(m.rot().log().unit())
print(w)

d = l_loc <= l_dir.unduale()

scene = Scene(children=[
        frame_mesh(),
#         line_mesh(l1.spin(m1)),
#         line_mesh(l1.spin(m1 * m1)),
#         line_mesh(l1.spin(m1 * m1 * m1)),
#         line_mesh(l1.spin(m1 * m1 * m1 * m1)),
#         line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1)),
#         line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1)),
#         line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 )),
#         line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 * m1)),
#         line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1)),
#         line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1)),
#         line_mesh(l1.spin(m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1)),
        
#         point_mesh(vsr.Vec(0,0,1), color='blue'),
#         point_mesh(vsr.Vec(1,0,1), color='red'),
        line_mesh(l, color='steelblue', length=100),
        line_mesh(l1, color='slategray', length=100),
#         point_mesh(l_loc, color=rgb2hex(255,56,0)),
#         point_mesh(vsr.Vec(0,0,0),color='#ffff00'),
#         point_mesh(p1,color='#ff00ff'),
#         vector_mesh(w, position=v),
#         vector_mesh(v),
        
        # l2 = l - l1
#         line_mesh(l2, color='hotpink', length=100),
        
        # l3 = l x l2 commutator
        line_mesh(l3, color='lightgreen', length=100),


#         vector_mesh(p1.vec(),color='#ff00ff'),
#         vector_mesh(l_dir,positil = l1.trs(vsr.Vec(0,0,1))on=p1,color='#ff00ff', arrow=True),


        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)

Dll: [ 0.7071 0 0 0 -0.7071 0.7071 ]
Vec: [ 0 0 1 ]


In [31]:
print(l3)

Dll: [ 0.7071 0 0 0 -0.7071 0.7071 ]


In [32]:
print(l)
print(l1)
print(l2)

Dll: [ 0 -0.7071 0.7071 -0.7071 0.7071 0.7071 ]
Dll: [ 0 0 1 0 0 0 ]
Dll: [ 0 -0.7071 -0.2929 -0.7071 0.7071 0.7071 ]


In [33]:
print(m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1 * m1)

Mot: [ 0.92 -0.38 0 0 -5.2e-18 0.38 -0.46 0.19 ]


In [34]:
l1 = vsr.Dll(vsr.Vec(0,0,0), vsr.Vec(1,0,0)).unit()
l2 = vsr.Dll(vsr.Vec(0,-1,1), vsr.Vec(1,-1,1)).unit()
pao = vsr.Vec(0,0,0).null()

l3 = l2 - l1
l4 = l2.comm(l1)

scene = Scene(children=[
        frame_mesh(),
        line_mesh(l1, color='#ffff00'),
        point_mesh(pao),
        vector_mesh(pao.vec()),
        
        line_mesh(l2, color='#ffff00'),
        
        line_mesh(l3, color='powderblue'), 
        line_mesh(l4, color='deeppink'),

        vector_mesh(l_dir, position=l_loc, color=rgb2hex(255,0,123), arrow=True),
        bivector_mesh((l_loc ^ l_dir), color = 'salmon'),

        
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'gray')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

In [35]:
print(l1)
print(l2)
print(l3)
print(l4)

Dll: [ 0 0 1 0 0 0 ]
Dll: [ 0.5774 0.5774 0.5774 0 0.5774 0.5774 ]
Dll: [ 0.5774 0.5774 -0.4226 0 0.5774 0.5774 ]
Dll: [ -0.5774 0.5774 0 0 -0.5774 0.5774 ]


In [36]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)
print(motor.trs())
print(motor.rot())

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]
Vec: [ 1 1 1 ]
Rot: [ 0.87 0 -0.5 0 ]


In [37]:
make_text?

In [38]:
@interact(x=10,y=10,z=10)
def position_camera(x,y,z):
    camera.position = [x,y,z]

In [39]:
class Sphere(object):
    def __init__(self):
        pass
    def update(vsr_sph):
        r, p, q = vsr_sph.rpo()
        
def make_sphere():
    return Mesh(geometry = SphereGeometry(),
                material= BasicMaterial(color='red'))

def make_circle():
    return Mesh(geometry = RingGeometry(thetaSegments=64), 
              material= BasicMaterial(color='black'))

class Game(object):
    class Circle(vsr.Cir, Mesh):
        def __init__(self):
            self._color = 'black'
            super(Mesh, self).__init__(geometry = RingGeometry(thetaSegments=64), 
                          material= BasicMaterial(color=self._color))
            
        @property
        def mesh(self):
            return self._mesh
        
        def update_mesh(self):
            (r, p, q) = self.rpo()
            self.geometry.innerRadius = r
            self.geometry.outerRadius = r + 0.1
            self.position = p
            self.quaternion = q
        

## Rotation

In [40]:

n1 = vsr.Vec(0,1,0).unit()
# n2 = vsr.Vec(-1,1,0).unit()
n2 = n1.spin(vsr.Biv(-np.deg2rad(30),0,0).exp())

B = n1 ^ n2

R = n1 * n2

a = vsr.Vec(1,0,1)
a_rot = a.reflect(n2).reflect(n1)
print(a_rot)
print(a.spin(R))


scene = Scene(children=[
        frame_mesh(),
        vector_mesh(n1,color='yellow'),
        vector_mesh(n2,color='cyan'),
        vector_mesh(a, color='hotpink'),
        vector_mesh(a.reflect(n2)),
        vector_mesh(a_rot, color='black'),
#         bivector_mesh(n1.unduale(),color='yellow'),
#         bivector_mesh(n2.unduale(),color='cyan'),
        bivector_mesh(B,color='gray'),
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

AttributeError: 'Vec' object has no attribute 'reflect'

In [ ]:
vsr.Vec(1,0,0).spin(R)

In [ ]:
2 * np.rad2deg(np.arccos(R[0]))

In [ ]:
sph1 = vsr.Sph(vsr.Vec(0.7,0,0).null(), 1.5)
sph2 = vsr.Sph(vsr.Vec(-0.5,0,0).null(), 1.)
cir = sph1.meet(sph2)
r = cir.rot()

sphere1 = Mesh(geometry = SphereGeometry(radius=sph1.radius(),
                                        widthSegments=64,
                                        heightSegments=64), 
              material= BasicMaterial(color='red'),
              position=np.array(sph1.pnt())[:3].tolist())

sphere2 = Mesh(geometry = SphereGeometry(radius=sph2.radius(),
                                        widthSegments=64,
                                        heightSegments=64), 
              material= BasicMaterial(color='blue', wireframe=False),
              position=np.array(sph2.pnt())[:3].tolist())

circle = Mesh(geometry = RingGeometry(innerRadius=cir.radius(),
                                                outerRadius=cir.radius()+0.05,
                                                thetaSegments=64), 
              material= BasicMaterial(color='black'),
              position=np.array(cir.pnt())[:3].tolist(),
              quaternion = cir.rot().quat().tolist())



scene = Scene(children=[sphere1, sphere2, circle])
c = PerspectiveCamera(position=[10, 5, 10])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'white')
display(renderer)
@interact(circlecolor=('hotpink', 'gray', 'black'),
         sphere1color=('hotpink', 'gray', 'black'),
         sphere2color=('hotpink', 'gray', 'black'))
def set_radius(circlecolor, sphere1color, sphere2color):
    sphere1.material.color = sphere1color
    sphere2.material.color = sphere2color
    circle.material.color = circlecolor

In [ ]:
%%javascript
console.log('hallo')
Jupyter.notebook.get_selected_cell()

In [ ]:
cir = vsr.Sph(vsr.Vec(0,0,0).null(), 3).meet(vsr.Sph(vsr.Vec(1,0,0).null(), 3))
print(cir.rpo())
radius, position, quat = cir.rpo()